In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [12]:
import pandas as pd
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
import random
import nltk
import re
#nltk.download('stopwords')
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
#from textblob import TextBlob
#from textblob.sentiments import NaiveBayesAnalyzer

In [13]:
np.random.seed(500)

Corpus = pd.read_csv("Data_hudoc_cases.csv",encoding='latin-1')
#label = list(np.random.randint(low = 2,size=11289))
#Corpus['label'] = label
# Step - a : Remove blank rows if any.
Corpus['Facts'].dropna(inplace=True)
Corpus = Corpus.applymap(str)

Corpus['label'] = ''
for i,x in Corpus.Unanimously.iteritems():
    is_violation = re.search('has been a violation of', x)
    not_violation = re.search('has been no violation of', x)
    if is_violation:
        Corpus['label'][i] = 1
    elif not_violation:
        Corpus['label'][i] = -1
    else:
        Corpus['label'][i] = 0
print(Corpus.head(100))
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['Facts'] = [entry.lower() for entry in Corpus['Facts']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['Facts']= [word_tokenize(entry) for entry in Corpus['Facts']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['Facts']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)
print(Corpus.head())

                                                  url language  \
0   https://hudoc.echr.coe.int/eng/#{"tabview":["d...  English   
1   https://hudoc.echr.coe.int/eng/#{"tabview":["d...  English   
2   https://hudoc.echr.coe.int/eng#{"itemid":["001...  English   
3   https://hudoc.echr.coe.int/eng#{"itemid":["001...  English   
4   https://hudoc.echr.coe.int/eng#{"itemid":["001...  English   
..                                                ...      ...   
95  https://hudoc.echr.coe.int/eng#{"itemid":["001...  English   
96  https://hudoc.echr.coe.int/eng/#{"tabview":["d...  English   
97  https://hudoc.echr.coe.int/eng/#{"tabview":["d...  English   
98  https://hudoc.echr.coe.int/eng/#{"tabview":["d...  English   
99  https://hudoc.echr.coe.int/eng#{"itemid":["001...  English   

                                                title  \
0   CASE OF NEGOVANOVIÄ AND OTHERS v. SERBIA (App...   
1   CASE OF BEG S.P.A. v. ITALY (Application no.  ...   
2   CASE OF ADVANCE PHARMA SP. Z O.O

In [14]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  79.08745247148289
